**Table of Contents**
1) Mission statment
2) Topics-Dynamic User Model\
    2.1) Why it does not represent the phanomena we deal with\
    2.2) Preferences change over time
3) Topics-Dynamic-Reverse User Model\
Preferences change over time
4) Baseline - Vanilla AutoRec\
    5.1) Multiple decision rules\
    5.2) MRSE\
    5.3) ARRI\
    5.4) Model Convergence
5) AutoRec with ratings history\
    6.1) Different temporal window size\
    6.2) Different initialization user-item-window tensor\
    6.3) MRSE\
    6.4) ARRI\
    6.5) Model Convergence
6) AutoRec with ratings history & recommendations history\
    7.1) Different temporal window size\
    7.2) Different initialization user-item-window tensor\
    7.3) MRSE\
    7.4) ARRI\
    7.5) Model Convergence
7) Topic-Dynamic-Reverse Recommender-Agnostic\
Breaking the recommender-environment feedback loop
8) Conclusion


**Section 1: Mission Statment**

Recommending items from a fixed-size set of items to a fixed-szie set of users is an open challenge for the following reasons (partial list): 
1) i.i.d principle violation:\
One of the basic assumptions in training a 'classic' predictive model is independence & identically distribution of the training data.\
As a consequnce, one assumes that all data points presented to the model during training are drawn from a time-invariant distribution function.\
But in Rec Systems, this is not hold. The users change their behaviors (their underlyning distributions) as a response to the recommender decisions.\
This forces the recommender to be strategic in the face of uncertainity, which is quite difficult during training, when the recommender is yet uncapable in term of predictive performance. 
2) Even regardless to the feedback loop between the recommender & the users, still the user model changes frequently over time during training.\
Since the recommender model (at least the vanilla version) does not have a ense of time, it is hardly capable learning the prefernces changing function, even when it is user-invariant, time-invariant and changes slow enough. 
3) We hypothesize that if a model would be capable to learn the users underlying change function, it could use this information for better predictions, which in turn would lead to better recommendations.
4) We examined various recommender models that were trained on a tensor of obeserved ratings over last k iterations rather than a single frameshot ratings matrix in the vanilla AutoRec version.
5) Furthermore, wed hypothesized that if we will feed the model with its previous decisions (another temporal window of recommendations), it will be able to mitigate the phanomena presented in 1).
TL;DR it didn't work though. 

**Section 2: Topics-Dynamic User-Model**

There are countless user models, aka, ways that the user can change their behavior over time.\
We choose to keep it simple, and stay close as possible to the Topics-Dynamic User-Model we saw in class.\
In this User-Model, all items are grouped into topics, and users prefer topics, rather than an individual items.\
The preferences of user $u$ for items $i$ of topic $k$
is initialized as $\pi(u, k) \sim U(0.5, 5.5)$,\
while the topic, $k$ of item $i$ is chosen randomly from the set of all topics.\
When user $u$ is recommended item $i$ it will rate the item as $r_t(u, i)=clip(\pi(u, k)+\epsilon)$ where $\epsilon \sim N(0,\sigma^2)$ represents exogenous noise not,
modeled by the simulation, and clip truncates values to be between $1$ and $5$.\
if at round $t$ user $u$ is recommended an item of topic $k$,\
then at the next round her preferences are updated to be:\
$\pi_{t+1}(u, k) ← clip(\pi_t(u, k) + a),$\
$\pi_{t+1}(u, k') ← clip(\pi_t(u, k') - \frac{a}{K-1}) \quad \forall k' \neq k$\

**Section 2.1: Why it does not represent the phanomena we are intrested**

As one could understand from the equation above, this User Model makes the users to like the recomended items.\
In other words, even if initially the users don't favor the topic of the recommended items, quickly they 'change thier minds' and start to like it.\
We think that this causes the ARRI for the dynamic enviroment to raise above the ARRI for the static environment, even without expoloration et all, whether the recommender has succeded to predict the preferences well or not.\
We want to see what happens when the recommendations **affect badly** on the user welfare.

![TOPICS_DYNAMIC_AUTO_ARRI](project_artifacts/env_d_eg_0_0/ARRI.png)\
Fig 1: Autorec with Topics-Dynamic env with change coeeficient 0 (Topics-Static) & 1 (Topics-Dynamic with slow drift).\
While the ARRI for the static version is around 4.0 at best, the ARRI for the dynamic version is around 4.75.\
Averaged over 10 repeats: the center line is the average, and the vertical lines are error bars (std). 


Fig 2: User preferences change over time (and don't change, for Topics-Static)

**Section 3: Topics-Dynamic Reverse User-Model**

To create a scenario which is intersting, we modified the User Model source code, and created a 'rebelling' version of the user.\
That is, a user which tends to dislike whatever the recommender recommeds him.\
Thed user preferences change equations are similarf to the previous but with a little change:\
$\pi_{t+1}(u, k) ← clip(\pi_t(u, k) - a),$\
$\pi_{t+1}(u, k') ← clip(\pi_t(u, k') + \frac{a}{K-1}) \quad \forall k' \neq k$\
**We would consider this as a baseline for our next experiments.**

![REVERSE_TOPICS_DYNAMIC_AUTO_ARRI](project_artifacts/env_dr_eg_0_0/ARRI.png)\
Fig 3: Reverse-Topics-Dynamic.\
ARRI for the environment with the preferences change now is lower than the static version.

![REVERSE_TOPICS_DYNAMIC_AUTO_EPS_025_ARRI](project_artifacts/env_dr_eg_0_25/ARRI.png)\
Fig 4: Reverse-Topics-Dynamic. Autorec recommender with Epsilon Greedy decision rule (0.25)\
ARRI is not changed much. It's not a matter of exploration, but fundumental with the dynamics of the environment. 

Fig 4: User preferences change over time for Reverse-Topics-Dynamic

**Section 3.1 - Intermediate observation/hypothesis:**

ARRI for the Reverse-Topics-Dynamic is lower than the static version, and roughly fluctuate around the default rating of the Model's users-items-ratings matrix, which is 3.\
We hypothesis that the recommender model do not succeed to learn the preferences **change function** due to a lack of temporal information. Note that the function is time-invariant, user-invariant.\
We think that if we would provide the recommender model some **temporal information** about the oveserved ratings arrived from the users, it would be able to learn the preferences drift, and recommend items accordignly.\
For example, a better prediction using the temporal data, combined with other-than greedy decision rule (epsilon greedy for instance), can help the recommender recommend better items, in the face of lower uncertanity in the future. 

![REVERSE_TOPICS_DYNAMIC_AUTO_RMSE](project_artifacts/env_dr_eg_0_0/RMSE.png)\
Fig 6: Reverse-Topics-Dynamic RMSE.\
While the model predictive error gets lower for the static version as the simulation continous, predictive error for the reverse-dynamic version gets higher.

**Section 5 - Vannila Autorec:**

As a baseline, we evaluated the regular Autorec recommender (https://arxiv.org/abs/2007.07224) with our new user model, different epsilon greedy values and averaged over 10 independent experiments.\
In short, Vanilla Autorec failed to increase its ARRI over the reverse-dynamic-env.

**Section 5.1 - Different Epsilon Greedy values:**

To show that the problem is not related to the decision rule, we evaluated the follwoing values: 0.0, 0.25, 0.5.\
TL;DR: this is not the problem.

![REVERSE_TOPICS_DYNAMIC_AUTO_RMSE](project_artifacts/env_dr_eg_0_0/ARRI.png)
![REVERSE_TOPICS_DYNAMIC_AUTO_RMSE](project_artifacts/env_dr_eg_0_25/ARRI.png)
![REVERSE_TOPICS_DYNAMIC_AUTO_RMSE](project_artifacts/env_dr_eg_0_5/ARRI.png)\
Fig 7: ARRI with epsilon greedy 0, 0.25, 0.5 (left to right).
While as expected, more exploration adds more variability to the results (higher std), it does not improve the overall expected ARRI.


**Section 5.2 - Predictive Errors:**

![REVERSE_TOPICS_DYNAMIC_AUTO_RMSE](project_artifacts/env_dr_eg_0_0/RMSE.png)
![REVERSE_TOPICS_DYNAMIC_AUTO_RMSE](project_artifacts/env_dr_eg_0_25/RMSE.png)
![REVERSE_TOPICS_DYNAMIC_AUTO_RMSE](project_artifacts/env_dr_eg_0_5/RMSE.png)\
Fig 8: RMSE with epsilon greedy 0, 0.25, 0.5 (left to right).\
In all cases, in contrast to the static version of the env, RMSE (predictive error) for the reverse dynamic env gets higher. 

**Section 5.3 - Model Convergence:**

To aassure that it is not a problem of the model's capacity to learn the users-items-ratings matrix, or any other possible failures of the AutoEncoder Neural net model to learn, we show the training loss of the model, for the last iteration of the simulation.

![REVERSE_TOPICS_DYNAMIC_AUTO_RMSE](project_artifacts/env_dr_eg_0_0/REC_LAST_STEP_LOSSES.png)
![REVERSE_TOPICS_DYNAMIC_AUTO_RMSE](project_artifacts/env_dr_eg_0_25/REC_LAST_STEP_LOSSES.png)
![REVERSE_TOPICS_DYNAMIC_AUTO_RMSE](project_artifacts/env_dr_eg_0_5/REC_LAST_STEP_LOSSES.png)\
Fig 9: Last iteration of the simulation training loss.\
These plots show that the model is capable to learn the users-items-ratings matrix (from capacity perspective or optimization algorithm).

**Section 7 - Topic-Dynamic-Reverse Recommender-Agnostic:**

We wondered wether the negative impact on the ARRI is due to the poor predictive performance, or is it because some negative effect of the recommender-environment feedback loop.\
As we spoke in class, the recommender decisions affects the data distribution, breaking the i.i.d assumption in the basis of all predictive models, hence can causes weird behaviors.\
To check and isolate this effect, we modified the Topics-Dynamics source code, to change randomly regardless the recommender decisions.\
So we still have a time-invariant, user-invariant preferences change function, but now it is also recommender-invariant as well.\
For each user, at reset, a single topic is chosen randomly. Then, each time this user receive a recommendation, regardless the recommended item, the chosen topic preference is being reduced, and the rest are being increased (like regular Reverse-Topic-dynamic).
We repeated the previous experiments with the new environment to see if the source for the reduced performence is a lack of temporal information (as previously assumed), or the feedback loop itself.

![REVERSE_TOPICS_DYNAMIC_REC_AGNO_AUTO_ARRI](project_artifacts/env_drra_eg_0_0/ARRI.png)
![REVERSE_TOPICS_DYNAMIC_REC_AGNO_AUTO_ARRI](project_artifacts/env_drra_eg_0_25/ARRI.png)
![REVERSE_TOPICS_DYNAMIC_REC_AGNO_AUTO_ARRI](project_artifacts/env_drra_eg_0_5/ARRI.png)\
Fig 10: Reverse-Topic-Dynamic Recommendation-Agnostic ARRI.\
We observe that the recommender succeed (kind of) to learn the preferences change function. ARRI for the dynamic version converges with the ARRI for the static version.


![REVERSE_TOPICS_DYNAMIC_REC_AGNO_AUTO_RMSE](project_artifacts/env_drra_eg_0_0/RMSE.png)
![REVERSE_TOPICS_DYNAMIC_REC_AGNO_AUTO_RMSE](project_artifacts/env_drra_eg_0_25/RMSE.png)
![REVERSE_TOPICS_DYNAMIC_REC_AGNO_AUTO_RMSE](project_artifacts/env_drra_eg_0_5/RMSE.png)\
Fig 11: Reverse-Topic-Dynamic Recommendation-Agnostic RMSE.\
Also from predictive error perspective: RMSE drops as the simulation continous.\
This in contrast to the equivalent environment with recommender-environment feedback loop (see section 5)


![REVERSE_TOPICS_DYNAMIC_REC_AGNO_AUTO_LOSS](project_artifacts/env_drra_eg_0_0/REC_LAST_STEP_LOSSES.png)
![REVERSE_TOPICS_DYNAMIC_REC_AGNO_AUTO_LOSS](project_artifacts/env_drra_eg_0_25/REC_LAST_STEP_LOSSES.png)
![REVERSE_TOPICS_DYNAMIC_REC_AGNO_AUTO_LOSS](project_artifacts/env_drra_eg_0_5/REC_LAST_STEP_LOSSES.png)\
Fig 12: Reverse-Topic-Dynamic Recommendation-Agnostic Last iteration loss.
